# Surface Event Analysis
###### This notebook analyzes surface event waveforms and calculates location, directivity, and velocity
###### Francesca Skene
###### fskene@uw.edu
###### Created: 7/22/22

Import Modules

In [ ]:
import obspy
import matplotlib.pyplot as plt
import numpy as np
from obspy.core import UTCDateTime
import pandas as pd
from obspy.clients.fdsn.client import Client
client = Client("IRIS")
from obspy.geodetics import *
import requests
import glob
import h5py
import sys
sys.path.append("/data/wsd01/pnwstore/")
from obspy.signal.cross_correlation import *
from mpl_toolkits import mplot3d
import tsfresh as tf
import scipy

from scipy import optimize
from scipy.optimize import curve_fit
from pnwstore.mseed import WaveformClient
from obspy.core.utcdatetime import UTCDateTime
client = WaveformClient()


Parameters

In [ ]:
t_before = 120 #number of seconds before pick time
t_after = 120 #number of seconds after pick time
fs = 40 #sampling rate that all waveforms are resampled to
window = 30 #window length of the signal
pr = 98 #percentile
thr = 7 #SNR threshold
station_distance_threshold = 25
pi = np.pi
v_s = 500 #shear wave velocity at the surface

## Define Functions

This functions cross correlates envelopes of waveforms to calculate picktimes

In [ ]:
def pick_time(ref_env, data_env_dict, est_picktimes):
    xcor = obspy.signal.cross_correlation.correlate(data_env_dict,ref_env,int(5*fs))
    index = np.argmax(xcor)
    cc = round(xcor[index],9) #correlation coefficient
    shift = 5*fs-index #how much it is shifted from the reference envelope
    #print(shift, cc, key)
    
    p = est_picktimes + shift/fs  # p is the new phase pick for each station
    return p

This function matches start_times between two lists to label waveforms with Event IDs

In [ ]:
def event_id(start_time, evt_id, st, t_before):
    for i,tr in enumerate(st): #Need to add t_before because the pick time is two minutes after the the trace_start_time
        est_picktimes.append(str(tr.stats.starttime + t_before)) 
    #Making list of strings for picktimes in order to match the format of start_time
    p_times_str = []
    for i in range(0, len(est_picktimes)):
        a = est_picktimes[i][:-1] #There is an extra 'Z' at the end of each string in est_picktimes so need to eliminate it
        p_times_str.append(a)

    start_time_set = set(start_time)
    p_times_set = set(p_times_str)
    try:
        c = list(p_times_set.intersection(start_time_set))[0] 

    # For the index of start_time in the file that matches the pick_time from the trace, give corresponding event ID
        for i in range(0, len(start_time)):
            if start_time[i]== c:
                event_ID = str(evt_id[i])
        return event_ID
    except:
        pass
        return 'no event ID match'

This function resamples the data in the streams to 40 Hz

In [ ]:
def resample(st, fs):
    for i in st:
        i.detrend(type='demean')
        i.taper(0.05)
        i.resample(fs)   
    return st

Function to fit data

In [ ]:
  def test_func(theta, a,theta0, c):
                    return a * np.cos(theta-theta0)+c

##  Import and organize metadata

### 1. Volcano Data (network and station, labeled with volcano name)

In [ ]:
#this data includes all stations within 50km of each volcano and the lat, lon, elev of each station
df = pd.read_csv('Volcano_Metadata_50km.csv')

Input Volcano Names and Locations

In [ ]:
#data obtained from www.lat-long.com
volc_lat_lon = {}
volc_lat_lon['Mt_Rainier'] = [46.8528857, -121.7603744, 4392.5]
volc_lat_lon['Mt_Adams'] = [46.202621, -121.4906384, 3743.2]
volc_lat_lon['Mt_Baker'] = [48.7773426,  -121.8132008, 3287.6]
volc_lat_lon['Mt_St_Helens'] = [46.1912, -122.1944, 2549]
volc_lat_lon['Glacier_Peak'] = [48.1112273, -121.1139922, 3213]
volc_lat_lon['Crater_Lake']=[42.907745, -122.143494, 1883]
volc_lat_lon['Mt_Hood']=[45.373221, -121.696509, 3428.7]
volc_lat_lon['Newberry']=[43.7220653, -121.2344654, 2435]

### 2. Station Data

Find Volcanoes within 100km of each station
Create dictionary of station metadata: network, station, latitude, longitude

In [ ]:
#Data obtained from IRIS
net_sta_dict = {}
url = 'https://service.iris.edu/fdsnws/station/1/query?net=CC,UW,UO&level=channel&format=text&includecomments=true&nodata=404'
r = requests.get(url)

#This for loop extrapolates the necessary data components from the url and adds them to the dictionary
for line in r.iter_lines():
    line = line.decode('utf-8')
    if ( "#" not in line ):
        net = line.split("|")[0]
        sta = line.split("|")[1]
        loc = line.split("|")[2]
        cha = line.split("|")[3]
        lat = float(line.split("|")[4])
        lon = float(line.split("|")[5])
        elev = line.split("|")[6] 
        netsta = net + "." + sta
        net_sta_dict[netsta] = [ net, sta, lat, lon]

### 3. Surface Event Data from PNSN

In [ ]:
#use this data to loop through

df3= pd.read_csv('../surface_events/PNSN_Pick_Label.csv')

net = df3["Network"].values.tolist()
sta = df3["Station"].values.tolist()
evt_id = df3['Event_ID'].values.tolist()
start_time = df3['Picktime'].values.tolist()
label = df3['Label'].values.tolist()

In [ ]:
test = df3[df3['Label']== 'su']['Event_ID'].values.tolist()
print(len(np.unique(test)))

For each volcano, determining a time series of events per month

In [ ]:
#Reading in surface event data from PNSN
dir1 = "/data/wsd01/PNSN_exotic/MLdataset_PNSN_non_earthquake_sources_MSEED/"
file_list = glob.glob(dir1 + "/*su*" )

#Splitting PNSN data into the wanted data: net, sta, cha, and time
NET = ([i.split('/')[-1].split(".")[0] for i in file_list])
STA = ([i.split('/')[-1].split(".")[1] for i in file_list])
CHA = ([i.split('/')[-1].split(".")[3] for i in file_list])
year = ([i.split('/')[-1].split(".")[4] for i in file_list])
month = ([i.split('/')[-1].split(".")[5] for i in file_list])
day = ([i.split('/')[-1].split(".")[6] for i in file_list])
hour = ([i.split('/')[-1].split(".")[7] for i in file_list])
minute = ([i.split('/')[-1].split(".")[8] for i in file_list])
second = ([i.split('/')[-1].split(".")[9] for i in file_list])
millisecond = ([i.split('/')[-1].split(".")[10] for i in file_list])
impulsivity = ([i.split('/')[-1].split(".")[12] for i in file_list])

## Calculating seasonal occurence of events

In [ ]:
for name in volc_lat_lon:
    events = []
    starttimes = []
    stas = []
    nets = []
    for i in range(0, len(start_time)):
        if label[i]=='su':
            try:
                #associates a volcano to each station based on 'Volcano_Metadata50km.csv'
                associated_volcano = df[df['Station']== sta[i]]['Volcano_Name'].values[0:]
            except: 
                associated_volcano = 'unknown'
                
            for volc in associated_volcano:
                if volc == name and evt_id[i]!=evt_id[i-1]:
                    events.append(evt_id[i])
                    starttimes.append(start_time[i])
                    stas.append(sta[i])
                    nets.append(net[i])
    print(len(starttimes))
    print(len(nets))
    print(len(stas))

    
    num_events = {}
    for year in range (1980, 2021):
        for month in range (1, 13):
            Nevt = []
            period = str(year)+"_"+str(month)
            t0 = UTCDateTime(year, month, 1)
            t1 = t0+3600*24*30
            for i in range(0, len(starttimes)):
                if t0<starttimes[i]<t1:
                    Nevt.append(events[i])
                    num_events[period]=len(Nevt)

    periods = list(num_events.keys())
    num_of_events = list(num_events.values())

    fig = plt.figure(figsize = (100, 10))
    plt.bar(periods,num_of_events, color ='b', width = 0.4)
    plt.xlabel("year_month")
    plt.ylabel("No. of events")
    plt.title("Number of surface events per month at" + str(name))
    plt.show()
    break

## Plotting and gathering waveforms based on pick times from all event picks

Putting it all into a for loop to run through all of the streams

In [ ]:
for n in range(2020,2021):
    print(a)
    a = str(nets[n]+'.'+stas[n])
    if a != 'CN.SNB':
        #Gather all of the waveforms for a specific starttime as a stream
        t = UTCDateTime(starttimes[n])
        print(t)
        #t = UTCDateTime(int(year[n]), int(month[n]), int(day[n]), int(hour[n]), int(minute[n]), int(second[n]), int(millisecond[n]))
        #Gather which station picked the event from the dataset
        reference = a
        try:
            #associates a volcano to each station based on 'Volcano_Metadata.csv'
            associated_volcano = df[df['Station']== stas[n]]['Volcano_Name'].values[0]
            print(associated_volcano)
        except: 
            associated_volcano = 'unknown'

        if associated_volcano == 'unknown':
            print('unknown')
            pass

        else:
            #If the station does have a volcano nearby, give the other stations within 100km of that Volcano and pull
            #waveforms for each station based on the given starttime of the pick
            #find Stations list labeled with the volcano as well as Latitude, Longitude, and Elevation
            stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
            print(stations)
            networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
            latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
            longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
            elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()
            volc_lat = volc_lat_lon[associated_volcano][0]
            volc_lon = volc_lat_lon[associated_volcano][1]
        
            bulk = [] 
        
            for m in range(0, len(networks)):
                bulk.append([networks[m], stations[m], '*', '*', t-t_before, t+t_after])
            st = client.get_waveforms_bulk(bulk)
            for tr in st:
                if tr.stats.channel[0:2] != 'BH' and tr.stats.channel[0:2] != 'EH' and tr.stats.channel[0:2] != 'HH':
                        st.remove(tr)
                        continue
                if len(tr.data)/tr.stats.sampling_rate < 239.9:
                    st.remove(tr)
    
        #resampling the data to 40Hz for each trace
            st = resample(st,fs)
            print(st)
            break
        #Finding the EventIDs
            est_picktimes = []
            event_ID = event_id(start_time, evt_id, st, t_before)

        #Plotting all traces for one event with channel z, SNR>10, and bandpasses between 2-15Hz
            SNR = []
            stas = []
            data_env_dict = {}

            fig = plt.figure(figsize = (20,30), dpi=80)
            plt.subplots_adjust(hspace = .4)
            fig.suptitle('evtID:UW'+ event_ID+associated_volcano)

            ax1 = plt.subplot(4,1,1)
            iplot = 0
            for g,x in enumerate(st):
                t = x.times()
                x.detrend(type = 'demean')
                x.filter('bandpass',freqmin=2.0,freqmax=19.0,corners=2,zerophase=True)

                net = x.stats.network
                sta = x.stats.station
                cha = x.stats.channel
                starttime = x.stats.starttime
                smooth_length = 2*fs

                signal_window = x.copy()
                noise_window = x.copy()
                signal_window.trim(starttime+t_before-1, starttime+t_before-1+window)
                noise_window.trim(starttime-window+t_before-10, starttime+t_before-10)

                SNR.append(20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
                               / np.percentile(np.abs(noise_window.data),pr))/np.log(10))

                if cha[-1] == 'Z' and SNR[g]>thr:
                    #enveloping the data
                    data_envelope = obspy.signal.filter.envelope(x.data[110*fs:140*fs])
                    data_envelope /= np.max(data_envelope)
                    data_envelope += iplot*1.5
                    data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
                    data_env_dict[net+'.'+sta]= data_envelope

                    ax1.plot(t[100*fs:175*fs],x.data[100*fs:175*fs]/np.max(np.abs(x.data))+iplot*1.5)
                    ax1.plot(t[110*fs:140*fs], data_envelope, color = 'k')
                    ax1.set_ylabel('Velocity (m/s)')
                    ax1.set_xlabel('time (seconds)')
                    #ax1.vlines(120, ymin = 0, ymax = 1.5*iplot, color = 'k')
                    ax1.set_xlim([100,175])
                    #plt.text(len(x.data)/fs, iplot*1.5, str(SNR[g]))
                    iplot = iplot+1

                    stas.append(x.stats.station)
                else:
                    st.remove(x)

            if len(st)>=4:
                pick_times = []
                ref_env = data_env_dict[reference]
                for key in data_env_dict:
                    p = pick_time(ref_env, data_env_dict[key], UTCDateTime(est_picktimes[0]))
                    pick_times.append(p)

                lats = []
                lons = []
                elevs = []
                for i in stas:
                    a = stations.index(i)
                    lats.append(latitudes[a])
                    lons.append(longitudes[a])
                    elevs.append(elevations[a])

                #calculating azimuth for each station with respect to the middle of the volcano
                azimuth = []
                for i in range(0, len(stas)):
                    lat2 = lats[i]
                    lon2 = lons[i]
                    lat1 = volc_lat_lon[associated_volcano][0]
                    lon1 = volc_lat_lon[associated_volcano][1]
                    a,b,c = (gps2dist_azimuth(lat1, lon1, lat2, lon2, a=6378137.0, f=0.0033528106647474805))
                    azimuth.append(b)
                

                #select station and component
                ax2 = plt.subplot(4,1,2)
                ax2.set_title('20 second time window')
                stations = stas
                channel = "*Z"
                spectra_method = "welch"
                char_freq_method = "mean"

                # read and preprocess data
                # which frequencies do I want the low_cut and high_cut to be?
                low_cut = 2
                high_cut = 12
                st.filter("bandpass",freqmin=low_cut,freqmax=high_cut)
                st.taper(max_percentage=0.01,max_length=20)
                st.trim(starttime=min(pick_times),endtime=min(pick_times)+20) 

                # make plot of spectra
                colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
                char_freq = []
                spectra_list = []
                for i in range(len(stations)):
                    data = st.select(station=stations[i],channel=channel)[0].data*1000
                    if spectra_method == "fft":
                        spectra = abs(np.fft.rfft(data))
                        f = np.fft.fftfreq(len(data), d=1/st[0].stats.sampling_rate)
                        f = f[:len(f)//2+1]
                        power = np.square(spectra)
                        psd = power/(f[1]-f[0])


                    elif spectra_method == "welch":
                        f,psd=scipy.signal.welch(data,fs=st[0].stats.sampling_rate,nperseg=81,noverlap=1)

                    # just get the indices of frequencies within the filter band
                    above_low_cut = [f>low_cut]
                    below_high_cut = [f<high_cut]
                    in_band = np.logical_and(above_low_cut,below_high_cut)[0]
                    f = f[in_band]
                    psd = psd[in_band]

                    ax2.plot(f,psd,label=stations[i],linewidth=2)
                    ax2.set_xscale('log')
                    ax2.set_yscale('log')
                    ax2.set_xlabel('Frequency [Hz]')
                    ax2.set_ylabel('PSD [$(mm/s)^2$/Hz]')
                    spectra_list.append(psd)
                    ax2.legend()

                    # calculate characteristic frequency and report
                    if char_freq_method == "max":
                        char_freq_max = f[np.argmax(psd)]
                    elif char_freq_method == "mean":
                        char_freq_mean= np.sum(psd*f)/np.sum(psd)
                    elif char_freq_method == "median":
                        psd_cumsum = np.cumsum(psd)
                        psd_sum = np.sum(psd)
                        char_freq_median = f[np.argmin(np.abs(psd_cumsum-psd_sum/2))]

                    char_freq.append(char_freq_mean)
                    ymax=np.max(psd)*10

        #             plt.vlines(char_freq_max, ymin=0, ymax = ymax, color = 'k')
                    plt.vlines(char_freq_mean, ymin=0, ymax = ymax, color = 'k')
        #             plt.vlines(char_freq_median, ymin=0, ymax = ymax, color = 'deeppink')
                    ax2.grid(True)

                #plotting azimuth versus characteristic frequency for each station
                ydata = char_freq
#                 xdata = []
#                 xdata2 = []
#                 for d in azimuth:
#                     xdata.append(np.deg2rad(d[1]))
#                     xdata2.append(d[0])     
#                 print(xdata)
                print(azimuth)
                #optimizing parameters to fit data to test_function
                params, params_covariance = optimize.curve_fit(test_func, np.deg2rad(azimuth), ydata, p0=None)
                print(params)
                
                #manipulating the data
                data = {'azimuth':azimuth, 'freq':ydata, 'station':stas}
                DF = pd.DataFrame(data, index = None)
                DF2 = DF.sort_values('azimuth')
                y_data =  DF2["freq"].values.tolist()
                x_data =  DF2["azimuth"].values.tolist()
                x_data = np.asarray(x_data)
                x_points = np.linspace(0,360, 100)
                print(x_data)
                #plotting the raw data and the fitted curve
                ax3 = plt.subplot(4,1,3)
                ax3.set_title('Fitting Sin curve')
                ax3.set_ylabel('characteristic frequency(Hz)')
                ax3.set_xlabel('azimuth(degrees)')
                ax3.scatter(x_data, y_data, label='Data')
                d = test_func(np.deg2rad(x_points), params[0], params[1], params[2])
                ax3.plot(x_points, d, label='Fitted function')
                ax3.plot(x_data,y_data, '--', label='rawdata')
                ax3.legend(loc='best')
                
#                 ax4= plt.subplot(4,1,4, polar=True)
#                 ax4.set_theta_offset(pi/2)
#                 r = []
#                 theta = []
#                 for y in azimuth:
#                     r.append(y[0])
#                     theta.append(360 - y[1])
#                 for t in theta:
#                     if t > 360:
#                         i = theta.index(t)
#                         theta = theta[:i]+[t-360]+theta[i+1:]       
#                 for i in range(0,len(r)):
#                     ax4.plot(np.deg2rad(theta[i]), r[i], 'g.')
#                     ax4.text(np.deg2rad(theta[i]),r[i],stas[i]) 
                
#                 len_r = int(max(r))
#                 line_length = np.linspace(0,len_r,len_r+1)
#                 direction=[]
#                 direction = [params[2] for i in range(len_r+1)] 
#                 ax4.plot(direction,line_length, 'k-')  #plot the estimated direction of the event
                
#                 #calculating velocity from the frequency shift
#                 fmax = max(d)
#                 fmin = min(d)
#                 v = v_s*((fmax-fmin)/(fmax+fmin))
#                 print(v,'m/s')

               # plt.savefig('evtID:UW'+ event_ID+associated_volcano+'.png')


Testing the correlation against fake data to ensure curve fitting is working

In [ ]:
y =  [1.0, 0.866025624917, 0.500000766025, 1.32679489668e-06, -0.499998467948, -0.86602429812, -0.999999999996, -0.866026951708, -0.500003064098, -3.98038469047e-06, 0.499996169868, 0.866022971317 ]
x = [ 0,30,60,90,120,150,180,210,240,270,300,330 ]


params, params_covariance = optimize.curve_fit(test_func, np.deg2rad(x), y, p0=None)


d = test_func(np.deg2rad(x), params[0], params[1], params[2])
plt.plot(x,y, label = 'raw data')
plt.plot(x,d,label = 'fitted function')